## 1. Анализ примера записи данных сигналов

In [53]:
# загрузка библиотеки для работы с json файлами
import json

In [54]:
# загрузка файла с данными сигналов
with open('data/example_kafka.json') as f:
    example_kafka = json.load(f)

In [55]:
# вывод ключей к данным 
example_kafka[0].keys()

dict_keys(['Timestamp', 'Topic', 'Partition', 'Offset', 'Key', 'Headers', 'Message'])

In [56]:
# вывод атрибутов значения сигналов
example_kafka[0]['Message'].keys()

dict_keys(['moment', 'SM_Exgauster\\[0:0]', 'SM_Exgauster\\[0:1]', 'SM_Exgauster\\[0:2]', 'SM_Exgauster\\[0:3]', 'SM_Exgauster\\[0:4]', 'SM_Exgauster\\[0:5]', 'SM_Exgauster\\[0:6]', 'SM_Exgauster\\[0:7]', 'SM_Exgauster\\[0:8]', 'SM_Exgauster\\[0:9]', 'SM_Exgauster\\[0:10]', 'SM_Exgauster\\[0:11]', 'SM_Exgauster\\[0:12]', 'SM_Exgauster\\[0:13]', 'SM_Exgauster\\[0:14]', 'SM_Exgauster\\[0:15]', 'SM_Exgauster\\[0:16]', 'SM_Exgauster\\[0:17]', 'SM_Exgauster\\[0:18]', 'SM_Exgauster\\[0:19]', 'SM_Exgauster\\[0:20]', 'SM_Exgauster\\[0:21]', 'SM_Exgauster\\[0:22]', 'SM_Exgauster\\[0:23]', 'SM_Exgauster\\[0:24]', 'SM_Exgauster\\[0:25]', 'SM_Exgauster\\[0:27]', 'SM_Exgauster\\[0:28]', 'SM_Exgauster\\[0:29]', 'SM_Exgauster\\[0:30]', 'SM_Exgauster\\[0:31]', 'SM_Exgauster\\[0:32]', 'SM_Exgauster\\[0:33]', 'SM_Exgauster\\[0:34]', 'SM_Exgauster\\[0:35]', 'SM_Exgauster\\[0:36]', 'SM_Exgauster\\[0:37]', 'SM_Exgauster\\[0:41]', 'SM_Exgauster\\[0:42]', 'SM_Exgauster\\[0:43]', 'SM_Exgauster\\[0:44]', 'SM_E

In [57]:
# количество атрибутов значения данных
len(example_kafka[0]['Message'].keys())

766

## 2. Подключение к базе данных

In [6]:
#pip install psycopg2

In [235]:
# загрузка библиотек
import psycopg2
from psycopg2.extras import Json

In [236]:
psycopg2.__version__

'2.9.5 (dt dec pq3 ext lo64)'

In [59]:
# загрузка конфигурации доступа к базе данных в облаке
with open('DATABASE_URL.txt', 'r') as f:
    DATABASE_URL = f.read()

In [230]:
# создание подключения к базе данны
connect = psycopg2.connect(DATABASE_URL)

In [231]:
# создание объекта курсора подключения к базе данных
cursor = connect.cursor()

In [62]:
def connect_db(URL):
    connect = psycopg2.connect(URL)
    cursor = connect.cursor()
    return connect, cursor    

In [46]:
# загрузка конфигурации доступа к базе данных в облаке
#with open('DATABASE_URL_2.txt', 'r') as f:
#    DATABASE_URL_2 = f.read()

In [47]:
# загрузка конфигурации доступа к базе данных в облаке
#with open('DATABASE_URL_3.txt', 'r') as f:
#    DATABASE_URL_3 = f.read()

In [48]:
#connect2, cursor2 = connect_db(DATABASE_URL_2)

In [49]:
#connect3, cursor3 = connect_db(DATABASE_URL_3)

In [338]:
# закрытие подключения к базе данных 'postgres'
connect.close()

## 3. Создание схемы Stagen - Storage Layer

In [69]:
# удаление схемы 'Stagen' если она существует 
#cursor.execute("DROP SCHEMA stg CASCADE;")
#connect.commit() 

In [73]:
# создание схемы 'Stagen' - Storage Layer
cursor.execute("CREATE SCHEMA stg;")
connect.commit()  

### 3.1 Создание таблицы данные сигналов - 'signal_messages_json'

In [74]:
# удаление таблицы 'signal_messages_json' если она существует 
cursor.execute("DROP TABLE IF EXISTS stg.signal_messages_json;")
connect.commit()  

In [75]:
# создание таблицы - 'signal_messages_json'
cursor.execute("CREATE TABLE stg.signal_messages_json( \
                message JSON NOT NULL);")
connect.commit()  

In [77]:
# значение атрибутов в таблице 'signal_messages_json'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'signal_messages_json' AND table_schema = 'stg';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['message']


In [78]:
# запись данных в таблицу - 'signal_messages_json'
cursor.execute("INSERT INTO stg.signal_messages_json( \
                message) \
                VALUES (%s);", [Json(example_kafka[0])])
connect.commit() 

In [83]:
# проверка записи данных в таблицу 'signal_messages_json'
cursor.execute("SELECT * FROM stg.signal_messages_json LIMIT 5;")
cursor.fetchall()

[({'Timestamp': '2022-12-30T06:00:00.592+00:00',
   'Topic': 'zsmk-9433-dev-01',
   'Partition': 0,
   'Offset': 62501,
   'Key': None,
   'Headers': {},
   'Message': {'moment': '2022-12-30T05:59:25.16886',
    'SM_Exgauster\\[0:0]': 1.2731481790542603,
    'SM_Exgauster\\[0:1]': 1.5856481790542603,
    'SM_Exgauster\\[0:2]': 0.9835540056228638,
    'SM_Exgauster\\[0:3]': 1.6203702688217163,
    'SM_Exgauster\\[0:4]': 2.060185194015503,
    'SM_Exgauster\\[0:5]': 1.1863425970077515,
    'SM_Exgauster\\[0:6]': 2.9177370071411133,
    'SM_Exgauster\\[0:7]': 1.9097222089767456,
    'SM_Exgauster\\[0:8]': 2.9585418701171875,
    'SM_Exgauster\\[0:9]': 1.3657407760620117,
    'SM_Exgauster\\[0:10]': 0.8391203880310059,
    'SM_Exgauster\\[0:11]': 0.8796296119689941,
    'SM_Exgauster\\[0:12]': 1.2442129850387573,
    'SM_Exgauster\\[0:13]': 1.359953761100769,
    'SM_Exgauster\\[0:14]': 0.5324074029922485,
    'SM_Exgauster\\[0:15]': 0.9780092239379883,
    'SM_Exgauster\\[0:16]': 1.730324

In [84]:
# вывод времени записи данных - 'Timestamp' и имя топика - 'Topic'
cursor.execute("SELECT message -> 'Timestamp' AS Timestamp, \
                       message -> 'Topic' AS Topic \
                FROM stg.signal_messages_json;")
cursor.fetchall()

[('2022-12-30T06:00:00.592+00:00', 'zsmk-9433-dev-01')]

In [85]:
# вывод времени сигнала - 'moment' и значение уставки для сигнала - 'SM_Exgauster\\[0:9]'
cursor.execute("SELECT message -> 'Message' ->>  'moment' AS moment, \
                       message -> 'Message' ->>  'SM_Exgauster\\[0:9]' AS SM_Exgauster_0_9 \
                FROM stg.signal_messages_json;")
cursor.fetchall()

[('2022-12-30T05:59:25.16886', '1.3657407760620117')]

In [86]:
# вывод всех данных по записи данных сигналов - 'Message'
cursor.execute("SELECT message -> 'Message' AS Message \
                FROM stg.signal_messages_json;")
cursor.fetchall()

[({'moment': '2022-12-30T05:59:25.16886',
   'SM_Exgauster\\[0:0]': 1.2731481790542603,
   'SM_Exgauster\\[0:1]': 1.5856481790542603,
   'SM_Exgauster\\[0:2]': 0.9835540056228638,
   'SM_Exgauster\\[0:3]': 1.6203702688217163,
   'SM_Exgauster\\[0:4]': 2.060185194015503,
   'SM_Exgauster\\[0:5]': 1.1863425970077515,
   'SM_Exgauster\\[0:6]': 2.9177370071411133,
   'SM_Exgauster\\[0:7]': 1.9097222089767456,
   'SM_Exgauster\\[0:8]': 2.9585418701171875,
   'SM_Exgauster\\[0:9]': 1.3657407760620117,
   'SM_Exgauster\\[0:10]': 0.8391203880310059,
   'SM_Exgauster\\[0:11]': 0.8796296119689941,
   'SM_Exgauster\\[0:12]': 1.2442129850387573,
   'SM_Exgauster\\[0:13]': 1.359953761100769,
   'SM_Exgauster\\[0:14]': 0.5324074029922485,
   'SM_Exgauster\\[0:15]': 0.9780092239379883,
   'SM_Exgauster\\[0:16]': 1.7303240299224854,
   'SM_Exgauster\\[0:17]': 1.2152777910232544,
   'SM_Exgauster\\[0:18]': 1.1689815521240234,
   'SM_Exgauster\\[0:19]': 1.3946759700775146,
   'SM_Exgauster\\[0:20]': 2.8

In [87]:
# значение атрибутов в данных сигналов - 'Message'
cursor.execute("SELECT json_object_keys (message ->'Message') \
                FROM stg.signal_messages_json;")
message_keys = cursor.fetchall()
message_keys[:7]

[('moment',),
 ('SM_Exgauster\\[0:0]',),
 ('SM_Exgauster\\[0:1]',),
 ('SM_Exgauster\\[0:2]',),
 ('SM_Exgauster\\[0:3]',),
 ('SM_Exgauster\\[0:4]',),
 ('SM_Exgauster\\[0:5]',)]

In [88]:
# проверка соответствия исходным данным
len(example_kafka[0]['Message'].keys()), len(message_keys)

(766, 766)

Данные записаны **без потерь**

In [89]:
# вывод наименований таблиц схемы 'Stagen'
cursor.execute("SELECT table_name FROM information_schema.tables \
             WHERE table_schema = 'stg';")
cursor.fetchall()

[('signal_messages_json',)]

# 4. Создание схемы Bl model - Storage Layer

In [90]:
# удаление схемы 'Bl model' если она существует 
#cursor.execute("DROP SCHEMA blm CASCADE;")
#connect.commit()

In [91]:
# создание схемы 'Bl model' - Storage Layer
cursor.execute("CREATE SCHEMA blm;")
connect.commit()  

## 4.1 Анализ данных уставок сигналов

In [92]:
# загрузка библиотек
import pandas as pd
import numpy as np

In [93]:
# загрузка данных уставок сигналов
signals_kafka  = pd.read_csv('data/signals_kafka.csv')
signals_kafka.head()

,place,type,comment,exhauster,active
0,SM_Exgauster\[0:0],analog,Горизонтал вибрация подшипника №1. Эксгаустер №3,3.0,1
1,SM_Exgauster\[0:1],analog,Вертикал вибрация подшипника №1. Эксгаустер №3,3.0,1
2,SM_Exgauster\[0:2],analog,Осевая вибрация подшипника №1. Эксгаустер №3,3.0,1
3,SM_Exgauster\[0:3],analog,Горизонтал вибрация подшипника №2. Эксгаустер №3,3.0,1
4,SM_Exgauster\[0:4],analog,Вертикал вибрация подшипника №2. Эксгаустер №3,3.0,1


In [94]:
# проверка на наличие пропусков
signals_kafka.isna().sum()

place         0
type          0
comment       0
exhauster    12
active        0
dtype: int64

Имеется **12** пропусков в данных **'exhauster'** - эксгаустера

In [95]:
# вывод строк с пропущенными значениями
signals_kafka[signals_kafka['exhauster'].isna()]

,place,type,comment,exhauster,active
770,SM_Exgauster\[6:0],analog,Разрежение в общем коллекторе,NaN,1
771,SM_Exgauster\[6:1],analog,Температура в общем коллекторе,NaN,1
772,SM_Exgauster\[6:2],analog,Скорость агломашины,NaN,1
773,SM_Exgauster\[6.0],digital,Работа агломашины №1,NaN,1
774,SM_Exgauster\[7:0],analog,Разрежение в общем коллекторе,NaN,1
775,SM_Exgauster\[7:1],analog,Температура в общем коллекторе,NaN,1
776,SM_Exgauster\[7:2],analog,Скорость агломашины,NaN,1
777,SM_Exgauster\[7.0],digital,Работа агломашины №2,NaN,1
778,SM_Exgauster\[8:0],analog,Разрежение в общем коллекторе,NaN,1
779,SM_Exgauster\[8:1],analog,Температура в общем коллекторе,NaN,1


Исходя из контектста значения уставок **'comment**' показатели **не относятся** к какому либо **эксгаустеру**

In [96]:
# заполнение пропусков
signals_kafka['exhauster'] = signals_kafka['exhauster'].fillna(-1)

In [97]:
# подтверждение гипотезы
signals_kafka['exhauster'].value_counts(dropna=False)

 1.0    129
 2.0    129
 3.0    128
 4.0    128
 5.0    128
 6.0    128
-1.0     12
Name: exhauster, dtype: int64

### 4.2 Создание таблицы уставки сигналов - 'signal_settings'

In [98]:
# удаление таблицы 'signal_settings' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.signal_settings;")
connect.commit()  

In [99]:
# атрибуты уставок сигналов
signals_kafka.columns

Index(['place', 'type', 'comment', 'exhauster', 'active'], dtype='object')

In [100]:
# создание таблицы - 'signal_settings'
cursor.execute("CREATE TABLE blm.signal_settings( \
                signal_settings_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                place TEXT UNIQUE NOT NULL, \
                type TEXT NOT NULL, \
                comment TEXT NOT NULL, \
                exhauster INTEGER, \
                active INTEGER NOT NULL);")
connect.commit()  

In [101]:
# значение атрибутов в таблице 'signal_settings'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'signal_settings' AND table_schema = 'blm';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['signal_settings_id', 'place', 'type', 'comment', 'exhauster', 'active']


In [103]:
# данные для записи в таблицу 'signal_settings'
signal_settings = signals_kafka.values
signal_settings[:5]

array([['SM_Exgauster\\[0:0]', 'analog',
        'Горизонтал вибрация подшипника №1. Эксгаустер №3', 3.0, 1],
       ['SM_Exgauster\\[0:1]', 'analog',
        'Вертикал вибрация подшипника №1. Эксгаустер №3', 3.0, 1],
       ['SM_Exgauster\\[0:2]', 'analog',
        'Осевая вибрация подшипника №1. Эксгаустер №3', 3.0, 1],
       ['SM_Exgauster\\[0:3]', 'analog',
        'Горизонтал вибрация подшипника №2. Эксгаустер №3', 3.0, 1],
       ['SM_Exgauster\\[0:4]', 'analog',
        'Вертикал вибрация подшипника №2. Эксгаустер №3', 3.0, 1]],
      dtype=object)

In [104]:
# запись данных в таблицу  - 'signal_settings'
cursor.executemany("INSERT INTO blm.signal_settings( \
                    place, type, comment, exhauster, active) \
                    VALUES (%s, %s, %s, %s, %s);", signal_settings)
connect.commit() 

In [105]:
# проверка записи данных в таблицу 'signal_settings'
cursor.execute("SELECT * FROM blm.signal_settings LIMIT 5;")
rows = cursor.fetchall()
cursor.execute("SELECT column_name, ordinal_position \
                FROM information_schema.columns \
                WHERE table_name = 'signal_settings' AND table_schema = 'blm' \
                ORDER BY ordinal_position;")
columnsname = [_[0] for _ in cursor.fetchall()]
df = pd.DataFrame(rows, columns=columnsname)
df

,signal_settings_id,place,type,comment,exhauster,active
0,1,SM_Exgauster\[0:0],analog,Горизонтал вибрация подшипника №1. Эксгаустер №3,3,1
1,2,SM_Exgauster\[0:1],analog,Вертикал вибрация подшипника №1. Эксгаустер №3,3,1
2,3,SM_Exgauster\[0:2],analog,Осевая вибрация подшипника №1. Эксгаустер №3,3,1
3,4,SM_Exgauster\[0:3],analog,Горизонтал вибрация подшипника №2. Эксгаустер №3,3,1
4,5,SM_Exgauster\[0:4],analog,Вертикал вибрация подшипника №2. Эксгаустер №3,3,1


In [106]:
# проверка записи данных в таблицу - 'signal_settings'
cursor.execute("SELECT COUNT(*) \
                FROM blm.signal_settings;")
cursor.fetchone()

(782,)

In [107]:
signals_kafka.shape

(782, 5)

## 4.3 Анализ маппинга сигналов

In [108]:
# загрузка исходного файла
xl = pd.ExcelFile('data/Маппинг сигналов.xlsx')

In [109]:
# вывод наименования листов
sheet_names = xl.sheet_names
sheet_names

['Эксгаустер № 1 (У-171)',
 'Эксгаустер № 2 (У-172)',
 'Эксгаустер № 3 (Ф-171)',
 'Эксгаустер № 4 (Ф-172)',
 'Эксгаустер № 5 (X-171)',
 'Эксгаустер № 6 (X-172)',
 'Лист1']

In [110]:
# загрузка данных маппинга 'Эксгаустер № 1 (У-171)'
mapping_signals = pd.read_excel('data/Маппинг сигналов.xlsx', sheet_name=0)
mapping_signals.head()

,Сигнал,Unnamed: 1,Unnamed: 2,Unnamed: 3,Код сигнала в Kafka,Название,Тип сигнала,Активность в ibaHD
0,Подшипник 1,Температура нагрева,Температура,temperature,SM_Exgauster\[2:27],Температура нагрева подшипника №1 (ротора прив...,analog,1
1,NaN,NaN,Уставки,alarm_max,SM_Exgauster\[2:65],Уставка аварийная max ТП т1 Эксгаустер №1,analog,1
2,NaN,NaN,NaN,alarm_min,SM_Exgauster\[2:74],Уставка аварийная min ТП т1 Эксгаустер №1,analog,1
3,NaN,NaN,NaN,warning_max,SM_Exgauster\[2:83],Уставка предупредительная max ТП т1 Эксгаустер №1,analog,1
4,NaN,NaN,NaN,warning_min,SM_Exgauster\[2:92],Уставка предупредительная min ТП т1 Эксгаустер №1,analog,1


In [111]:
# заполнение пропусков предыдущим значением
mapping_signals = mapping_signals.fillna(method='ffill')
mapping_signals.head()

,Сигнал,Unnamed: 1,Unnamed: 2,Unnamed: 3,Код сигнала в Kafka,Название,Тип сигнала,Активность в ibaHD
0,Подшипник 1,Температура нагрева,Температура,temperature,SM_Exgauster\[2:27],Температура нагрева подшипника №1 (ротора прив...,analog,1
1,Подшипник 1,Температура нагрева,Уставки,alarm_max,SM_Exgauster\[2:65],Уставка аварийная max ТП т1 Эксгаустер №1,analog,1
2,Подшипник 1,Температура нагрева,Уставки,alarm_min,SM_Exgauster\[2:74],Уставка аварийная min ТП т1 Эксгаустер №1,analog,1
3,Подшипник 1,Температура нагрева,Уставки,warning_max,SM_Exgauster\[2:83],Уставка предупредительная max ТП т1 Эксгаустер №1,analog,1
4,Подшипник 1,Температура нагрева,Уставки,warning_min,SM_Exgauster\[2:92],Уставка предупредительная min ТП т1 Эксгаустер №1,analog,1


In [112]:
# переименование атрибутов
mapping_signals.columns = ['Механизм', 'Сигнал', 'Показатель', 'Измерение', \
       'Код_сигнала_в_Kafka', 'Название', 'Тип_сигнала', 'Активность_в_ibaHD']

In [113]:
# проверка на наличие пропусков
mapping_signals.isna().sum()

Механизм               0
Сигнал                 0
Показатель             0
Измерение              0
Код_сигнала_в_Kafka    0
Название               0
Тип_сигнала            0
Активность_в_ibaHD     0
dtype: int64

In [114]:
# уникальные значения атрибутов
mapping_signals[['Механизм', 'Сигнал', 'Показатель', 'Измерение']].apply(pd.unique)

Механизм      [Подшипник 1, Подшипник 2, Подшипник 3, Подшип...
Сигнал             [Температура нагрева, Вибрация, Масло, Вода]
Показатель    [Температура, Уставки, Осевая, Горизонтальная,...
Измерение     [temperature, alarm_max, alarm_min, warning_ma...
dtype: object

In [115]:
# количество уникальных значений по атрибутам
mapping_signals[['Механизм', 'Сигнал', 'Показатель', 'Измерение']].apply(lambda x: x.nunique())

Механизм      15
Сигнал         4
Показатель     5
Измерение     21
dtype: int64

In [119]:
# отбор показателей для парсинга
mapping_signals['Показатель'].unique()

array(['Температура', 'Уставки', 'Осевая', 'Горизонтальная',
       'Вертикальная'], dtype=object)

In [121]:
# отбор показателей для парсинга
mapping_signals.query("Показатель != 'Уставки'")['Код_сигнала_в_Kafka']

0      SM_Exgauster\[2:27]
5       SM_Exgauster\[2:2]
10      SM_Exgauster\[2:0]
15      SM_Exgauster\[2:1]
20     SM_Exgauster\[2:28]
25      SM_Exgauster\[2:5]
30      SM_Exgauster\[2:3]
35      SM_Exgauster\[2:4]
40     SM_Exgauster\[2:29]
45     SM_Exgauster\[2:30]
50     SM_Exgauster\[2:31]
55     SM_Exgauster\[2:32]
60     SM_Exgauster\[2:33]
65      SM_Exgauster\[2:8]
70      SM_Exgauster\[2:6]
75      SM_Exgauster\[2:7]
80     SM_Exgauster\[2:34]
85     SM_Exgauster\[2:11]
90      SM_Exgauster\[2:9]
95     SM_Exgauster\[2:10]
100    SM_Exgauster\[2:35]
Name: Код_сигнала_в_Kafka, dtype: object

## 4.4 Создание таблицы эксгаустеры - 'exhausters'

In [122]:
# создание таблицы - 'exhausters'
cursor.execute("CREATE TABLE blm.exhausters( \
                exhauster_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                exhauster_name TEXT UNIQUE NOT NULL);")
connect.commit() 

In [123]:
# данные для записи в таблицу 'exhausters'
exhausters = np.array(sheet_names[:-1]).reshape(-1,1)
exhausters

array([['Эксгаустер № 1 (У-171)'],
       ['Эксгаустер № 2 (У-172)'],
       ['Эксгаустер № 3 (Ф-171)'],
       ['Эксгаустер № 4 (Ф-172)'],
       ['Эксгаустер № 5 (X-171)'],
       ['Эксгаустер № 6 (X-172)']], dtype='<U22')

In [124]:
# запись данных в таблицу  - 'exhausters'
cursor.executemany("INSERT INTO blm.exhausters( \
                    exhauster_name) \
                    VALUES (%s);", exhausters)
connect.commit() 

In [125]:
# проверка записи данных в таблицу 'exhausters'
cursor.execute("SELECT * FROM blm.exhausters;")
cursor.fetchall()

[(1, 'Эксгаустер № 1 (У-171)'),
 (2, 'Эксгаустер № 2 (У-172)'),
 (3, 'Эксгаустер № 3 (Ф-171)'),
 (4, 'Эксгаустер № 4 (Ф-172)'),
 (5, 'Эксгаустер № 5 (X-171)'),
 (6, 'Эксгаустер № 6 (X-172)')]

## 4.5 Создание таблицы механизмы - 'mechanisms'

In [126]:
# создание таблицы - 'mechanisms'
cursor.execute("CREATE TABLE blm.mechanisms( \
                mechanism_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                mechanism_name TEXT UNIQUE NOT NULL);")
connect.commit() 

In [127]:
# данные для записи в таблицу 'mechanisms'
mechanism_names = mapping_signals[['Механизм']].drop_duplicates().values
mechanism_names[:5]

array([['Подшипник 1'],
       ['Подшипник 2'],
       ['Подшипник 3'],
       ['Подшипник 4'],
       ['Подшипник 5']], dtype=object)

In [128]:
# запись данных в таблицу  - 'mechanisms'
cursor.executemany("INSERT INTO blm.mechanisms( \
                    mechanism_name) \
                    VALUES (%s);", mechanism_names)
connect.commit() 

In [129]:
# проверка записи данных в таблицу 'mechanisms'
cursor.execute("SELECT * FROM blm.mechanisms;")
cursor.fetchall()

[(1, 'Подшипник 1'),
 (2, 'Подшипник 2'),
 (3, 'Подшипник 3'),
 (4, 'Подшипник 4'),
 (5, 'Подшипник 5'),
 (6, 'Подшипник 6'),
 (7, 'Подшипник 7'),
 (8, 'Подшипник 8'),
 (9, 'Подшипник 9'),
 (10, 'Охладитель'),
 (11, 'Газовый коллектор'),
 (12, 'Положение задвижки'),
 (13, 'Главный привод'),
 (14, 'Маслосистема'),
 (15, 'Работа эксгаустера')]

## 4.6 Создание таблицы сигналы - 'signals'

In [130]:
# удаление таблицы 'mechanisms' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.signals;")
connect.commit() 

In [131]:
# создание таблицы - 'mechanisms'
cursor.execute("CREATE TABLE blm.signals( \
                 signal_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                 signal_name TEXT UNIQUE NOT NULL);")
connect.commit()

In [132]:
# данные для записи в таблицу 'signals'
signal_names = mapping_signals[['Сигнал']].drop_duplicates().values
signal_names[:5]

array([['Температура нагрева'],
       ['Вибрация'],
       ['Масло'],
       ['Вода']], dtype=object)

In [133]:
# запись данных в таблицу - 'signals'
cursor.executemany("INSERT INTO blm.signals( \
                    signal_name) \
                    VALUES (%s);", signal_names)
connect.commit()

In [134]:
# проверка записи данных в таблицу 'mechanisms'
cursor.execute("SELECT * FROM blm.signals;")
cursor.fetchall()

[(1, 'Температура нагрева'), (2, 'Вибрация'), (3, 'Масло'), (4, 'Вода')]

## 4.7 Создание таблицы показатели - 'marks'

In [135]:
# создание таблицы - 'marks'
cursor.execute("CREATE TABLE blm.marks( \
                 mark_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                 mark_name TEXT UNIQUE NOT NULL);")
connect.commit()

In [136]:
# данные для записи в таблицу 'marks'
mark_names = mapping_signals[['Показатель']].drop_duplicates().values
mark_names[:5]

array([['Температура'],
       ['Уставки'],
       ['Осевая'],
       ['Горизонтальная'],
       ['Вертикальная']], dtype=object)

In [137]:
# запись данных в таблицу  - 'marks'
cursor.executemany("INSERT INTO blm.marks( \
                    mark_name) \
                    VALUES (%s);", mark_names)
connect.commit()

In [138]:
# проверка записи данных в таблицу 'marks'
cursor.execute("SELECT * FROM blm.marks;")
cursor.fetchall()

[(1, 'Температура'),
 (2, 'Уставки'),
 (3, 'Осевая'),
 (4, 'Горизонтальная'),
 (5, 'Вертикальная')]

## 4.8 Создание таблицы измерения - 'measurings'

In [139]:
# создание таблицы - 'measurings'
cursor.execute("CREATE TABLE blm.measurings( \
                 measuring_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                 measuring_name TEXT UNIQUE NOT NULL);")
connect.commit()

In [140]:
# данные для записи в таблицу 'measurings'
measuring_names = mapping_signals[['Измерение']].drop_duplicates().values
measuring_names[:5]

array([['temperature'],
       ['alarm_max'],
       ['alarm_min'],
       ['warning_max'],
       ['warning_min']], dtype=object)

In [141]:
# запись данных в таблицу - 'measurings'
cursor.executemany("INSERT INTO blm.measurings( \
                    measuring_name) \
                    VALUES (%s);", measuring_names)
connect.commit()

In [142]:
# проверка записи данных в таблицу 'measurings'
cursor.execute("SELECT * FROM blm.measurings LIMIT 5;")
cursor.fetchall()

[(1, 'temperature'),
 (2, 'alarm_max'),
 (3, 'alarm_min'),
 (4, 'warning_max'),
 (5, 'warning_min')]

In [143]:
# проверка соответствия данных на уникальность

In [144]:
cursor.execute("SELECT COUNT(DISTINCT(place)), COUNT(place) FROM blm.signal_settings;")
cursor.fetchall()

[(782, 782)]

In [145]:
mapping_signals['Код_сигнала_в_Kafka'].nunique(), mapping_signals['Код_сигнала_в_Kafka'].shape

(121, (121,))

In [146]:
# получение индекса сигнала 'signal_id' для случайного значения 'Код_сигнала_в_Kafka'
cursor.execute("SELECT signal_settings_id FROM blm.signal_settings WHERE place = (%s);", \
               (mapping_signals['Код_сигнала_в_Kafka'].sample(1).item(),))
cursor.fetchone()[0]

284

In [147]:
# получение индекса сигнала 'signal_id' для случайного значения 'Код_сигнала_в_Kafka'
cursor.execute("SELECT * FROM blm.signal_settings WHERE signal_settings_id = 284;")
cursor.fetchone()

(284,
 'SM_Exgauster\\[2:28]',
 'analog',
 'Температура нагрева подшипника №2 (ротора привода эксгаустера №1)',
 1,
 1)

In [148]:
# проверка соответствия данных
mapping_signals.query("Код_сигнала_в_Kafka == 'SM_Exgauster\\[2:147]'")

,Механизм,Сигнал,Показатель,Измерение,Код_сигнала_в_Kafka,Название,Тип_сигнала,Активность_в_ibaHD
96,Подшипник 8,Вибрация,Уставки,alarm_max,SM_Exgauster\[2:147],Уставка аварийная max ВВ 8т Эксгаустер №1,analog,1


## 4.9 Создание таблицы маппинг сигналов - 'mapping_signals'

In [149]:
# значение атрибутов в исходном файле
mapping_signals.columns

Index(['Механизм', 'Сигнал', 'Показатель', 'Измерение', 'Код_сигнала_в_Kafka',
       'Название', 'Тип_сигнала', 'Активность_в_ibaHD'],
      dtype='object')

In [150]:
# создание таблицы - 'mapping_signals'
cursor.execute("CREATE TABLE blm.mapping_signals( \
                 mapping_signals_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, \
                 mechanism_id INTEGER NOT NULL, \
                 signal_id INTEGER NOT NULL, \
                 mark_id INTEGER NOT NULL, \
                 measuring_id INTEGER NOT NULL, \
                 signal_settings_id INTEGER NOT NULL);")
connect.commit()

In [151]:
# функция записи данных в таблицу - 'mapping_signals'
def insert_mapping_signals(data):
    
    mechanism_name, signal_name, mark_name, measuring_name, place = data     
    cursor.execute("SELECT mechanism_id FROM blm.mechanisms WHERE mechanism_name = (%s);", \
               (mechanism_name,))
    mechanism_id = cursor.fetchone()[0]
    cursor.execute("SELECT signal_id FROM blm.signals WHERE signal_name = (%s);", \
               (signal_name,))
    signal_id = cursor.fetchone()[0]
    cursor.execute("SELECT mark_id FROM blm.marks WHERE mark_name = (%s);", \
               (mark_name,))
    mark_id = cursor.fetchone()[0]
    cursor.execute("SELECT measuring_id FROM blm.measurings WHERE measuring_name = (%s);", \
               (measuring_name,))
    measuring_id = cursor.fetchone()[0]
    cursor.execute("SELECT signal_settings_id FROM blm.signal_settings WHERE place = (%s);", \
               (place,))
    signal_settings_id = cursor.fetchone()[0]
    cursor.execute("INSERT INTO blm.mapping_signals( \
                    mechanism_id, signal_id, mark_id, measuring_id, signal_settings_id) \
                    VALUES (%s, %s, %s, %s, %s);", \
                    (mechanism_id, signal_id, mark_id, measuring_id, signal_settings_id))
    connect.commit()

In [153]:
%%time
# запись данных в таблицу - 'mapping_signals'
for i, row in mapping_signals.iloc[:,:5].iterrows():
    data = row.tolist()
    insert_mapping_signals(data)

Wall time: 9min 38s


In [155]:
# проверка записи данных в таблицу 'mapping_signals'
cursor.execute("SELECT * FROM blm.mapping_signals LIMIT 3;")
cursor.fetchall()

[(1, 1, 1, 1, 1, 283), (2, 1, 1, 2, 2, 321), (3, 1, 1, 2, 3, 330)]

In [156]:
# проверка записи данных в таблицу 'mapping_signals'
cursor.execute("SELECT COUNT(*) FROM blm.mapping_signals;")
cursor.fetchone()

(121,)

In [157]:
mapping_signals.shape

(121, 8)

In [158]:
# функция записи данных в таблицу - 'mapping_signals'
def open_data_xlsx(path, sheet_name):
    
    mapping_signals = pd.read_excel(path, sheet_name=sheet_name, usecols=[0, 1, 2, 3, 4])
    mapping_signals = mapping_signals.fillna(method='ffill')
    mapping_signals.columns = ['Механизм', 'Сигнал', 'Показатель', \
                               'Измерение', 'Код_сигнала_в_Kafka']
    for i, row in mapping_signals.iterrows():
        data = row.tolist()
        mechanism_name, signal_name, mark_name, measuring_name, place = data
        try:
            cursor.execute("SELECT mechanism_id FROM blm.mechanisms WHERE mechanism_name = (%s);", \
               (mechanism_name,))
            mechanism_id = cursor.fetchone()[0]
        except:
            # запись данных в таблицу  - 'mechanisms'
            cursor.execute("INSERT INTO blm.mechanisms( \
                            mechanism_name) \
                            VALUES (%s);", (mechanism_names,))
            connect.commit()
            cursor.execute("SELECT mechanism_id FROM blm.mechanisms WHERE mechanism_name = (%s);", \
               (mechanism_name,))
            mechanism_id = cursor.fetchone()[0]
        try:
            cursor.execute("SELECT signal_id FROM blm.signals WHERE signal_name = (%s);", \
               (signal_name,))
            signal_id = cursor.fetchone()[0]
        except:    
            # запись данных в таблицу - 'signals'
            cursor.execute("INSERT INTO blm.signals( \
                            signal_name) \
                            VALUES (%s);", (signal_names,))
            connect.commit()
            cursor.execute("SELECT signal_id FROM blm.signals WHERE signal_name = (%s);", \
               (signal_name,))
            signal_id = cursor.fetchone()[0]
        try:
            cursor.execute("SELECT mark_id FROM blm.marks WHERE mark_name = (%s);", \
               (mark_name,))
            mark_id = cursor.fetchone()[0]
        except:    
            # запись данных в таблицу  - 'marks'
            cursor.execute("INSERT INTO blm.marks( \
                            mark_name) \
                            VALUES (%s);", (mark_names,))
            connect.commit()
            cursor.execute("SELECT mark_id FROM blm.marks WHERE mark_name = (%s);", \
               (mark_name,))
            mark_id = cursor.fetchone()[0]
        try:
            cursor.execute("SELECT measuring_id FROM blm.measurings WHERE measuring_name = (%s);", \
               (measuring_name,))
            measuring_id = cursor.fetchone()[0]
        except:    
            # запись данных в таблицу - 'measurings'
            cursor.execute("INSERT INTO blm.measurings( \
                            measuring_name) \
                            VALUES (%s);", (measuring_names,))
            connect.commit()
            cursor.execute("SELECT measuring_id FROM blm.measurings WHERE measuring_name = (%s);", \
                           (measuring_name,))
            measuring_id = cursor.fetchone()[0]
            
        cursor.execute("SELECT signal_settings_id FROM blm.signal_settings WHERE place = (%s);", \
               (place,))
        signal_settings_id = cursor.fetchone()[0]
        
        cursor.execute("INSERT INTO blm.mapping_signals( \
                    mechanism_id, signal_id, mark_id, measuring_id, signal_settings_id) \
                    VALUES (%s, %s, %s, %s, %s);", \
                    (mechanism_id, signal_id, mark_id, measuring_id, signal_settings_id))
        connect.commit()

In [161]:
# путь к файлу с данными маппинга сигналов
path = 'data/Маппинг сигналов.xlsx'

In [162]:
%%time
# запись данных в таблицу - 'mapping_signals'
for sheet_name in sheet_names[1:-1]:
    open_data_xlsx(path, sheet_name)

Wall time: 29min 25s


In [163]:
# проверка записи данных в таблицу 'mapping_signals'
cursor.execute("SELECT COUNT(*) FROM blm.mapping_signals;")
cursor.fetchone()

(726,)

In [164]:
# вывод наименования листов
sheet_names

['Эксгаустер № 1 (У-171)',
 'Эксгаустер № 2 (У-172)',
 'Эксгаустер № 3 (Ф-171)',
 'Эксгаустер № 4 (Ф-172)',
 'Эксгаустер № 5 (X-171)',
 'Эксгаустер № 6 (X-172)',
 'Лист1']

In [165]:
# проверка записи данных в таблицу 'mapping_signals'
count = 0
for sheet in sheet_names[:-1]:
    mapping_signals = pd.read_excel(path, sheet_name=sheet, usecols=[0, 1, 2, 3, 4])
    count += mapping_signals.shape[0]
count

726

In [166]:
# отбор показателей для парсинга
cursor.execute("SELECT DISTINCT(mark_name) \
                FROM blm.mapping_signals \
                INNER JOIN blm.marks USING(mark_id);")
cursor.fetchall()

[('Вертикальная',),
 ('Уставки',),
 ('Горизонтальная',),
 ('Осевая',),
 ('Температура',)]

In [170]:
# отбор показателей для парсинга
cursor.execute("SELECT place \
                FROM blm.mapping_signals \
                INNER JOIN blm.marks USING(mark_id) \
                INNER JOIN blm.signal_settings USING(signal_settings_id) \
                WHERE mark_name != 'Уставки';")
place_list = cursor.fetchall()

In [171]:
# количество атрибутов сигнала
len(place_list)

126

In [174]:
# извлечение значений
place_list = [_[0] for _ in place_list]
place_list[:5]

['SM_Exgauster\\[2:27]',
 'SM_Exgauster\\[2:2]',
 'SM_Exgauster\\[2:0]',
 'SM_Exgauster\\[2:1]',
 'SM_Exgauster\\[2:28]']

In [186]:
# вывод атрибутов значения сигналов и преобразование в массив
places = np.array(list(example_kafka[0]['Message'].keys()))
places[:10]

array(['moment', 'SM_Exgauster\\[0:0]', 'SM_Exgauster\\[0:1]',
       'SM_Exgauster\\[0:2]', 'SM_Exgauster\\[0:3]',
       'SM_Exgauster\\[0:4]', 'SM_Exgauster\\[0:5]',
       'SM_Exgauster\\[0:6]', 'SM_Exgauster\\[0:7]',
       'SM_Exgauster\\[0:8]'], dtype='<U20')

In [187]:
# преобразование в DataFrame
places_df = pd.DataFrame(places.reshape(-1, 1), columns=['place'])
places_df.head()

,place
0,moment
1,SM_Exgauster\[0:0]
2,SM_Exgauster\[0:1]
3,SM_Exgauster\[0:2]
4,SM_Exgauster\[0:3]


In [190]:
# индексы атрибутов для парсинга
place_index = np.where(places_df['place'].isin(place_list))
place_index[:6]

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  27,  28,
         29,  30,  31,  32,  33,  34,  35,  40,  41,  42,  44,  45,  46,
         47,  48,  49, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
        264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
        277, 280, 281, 282, 283, 284, 285, 286, 287, 288, 293, 294, 295,
        297, 298, 299, 300, 301, 302, 485, 486, 487, 488, 489, 490, 491,
        492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504,
        505, 506, 507, 508, 511, 512, 513, 514, 515, 516, 517, 518, 519,
        524, 525, 526, 528, 529, 530, 531, 532, 533], dtype=int64),)

## 4.10 Создание таблицы календарь - 'calendar'

In [194]:
# вывод ключей к данным 
example_kafka[0].keys()

dict_keys(['Timestamp', 'Topic', 'Partition', 'Offset', 'Key', 'Headers', 'Message'])

In [197]:
list(example_kafka[0]['Message'].keys())[:5]

['moment',
 'SM_Exgauster\\[0:0]',
 'SM_Exgauster\\[0:1]',
 'SM_Exgauster\\[0:2]',
 'SM_Exgauster\\[0:3]']

In [232]:
# удаление таблицы 'calendar' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.calendar;")
connect.commit()  

In [233]:
# создание таблицы - 'calendar' UNIQUE
cursor.execute("CREATE TABLE blm.calendar( \
                 off_set INTEGER PRIMARY KEY, \
                 timestamp TEXT NOT NULL, \
                 moment DATE NOT NULL);")
connect.commit()

## 4.11 Создание таблицы показателей сигналов -  'moment_signals'

In [191]:
# вывод ключей к данным 
example_kafka[0].keys()

dict_keys(['Timestamp', 'Topic', 'Partition', 'Offset', 'Key', 'Headers', 'Message'])

In [192]:
# атрибуты эксгаутеров
#SM_Exgauster = ['SM_Exgauster_' + i.split('\\')[-1][1:-1].replace(':', '_').replace('.', '') 
#                + ' NUMERIC'
#                for i in list(example_kafka[0]['Message'].keys())[1:]]
#SM_Exgauster[:4]

In [226]:
# удаление таблицы 'moment_signals' если она существует 
cursor.execute("DROP TABLE IF EXISTS blm.moment_signals;")
connect.commit()  

In [198]:
# запрос на  создание таблицы - 'moment_signals'
#query = f"""CREATE TABLE blm.moment_signals( \
#                 off_set INTEGER PRIMARY KEY, \
#                 signal_settings_id INTEGER NOT NULL, \
#                 moment DATE NOT NULL, {", ".join(map(str, SM_Exgauster))});"""

In [246]:
# создание таблицы - 'moment_signals'
#cursor.execute(query)
#connect.commit()

In [227]:
# создание таблицы - 'moment_signals'
cursor.execute("CREATE TABLE blm.moment_signals( \
                 off_set INTEGER NOT NULL, \
                 signal_settings_id INTEGER NOT NULL, \
                 value NUMERIC NOT NULL);")
connect.commit()

In [228]:
# значение атрибутов в таблице 'moment_signals'
cursor.execute("SELECT column_name \
             FROM information_schema.columns \
             WHERE table_name = 'moment_signals' AND table_schema = 'blm';")
columnsname = [_[0] for _ in cursor.fetchall()]
print(columnsname)

['off_set', 'signal_settings_id', 'value']


In [280]:
# запись в таблицу 'moment_signals'
#cursor.execute(f"INSERT INTO blm.moment_signals VALUES ({str(moment_signals)[1:-1]});")
#connect.commit()

In [201]:
# проверка записи данных в таблицу 'moment_signals'
#cursor.execute("SELECT * FROM blm.moment_signals;")
#cursor.fetchone()

In [293]:
# функция записи данных в таблицу 'moment_signals'
def insert_moment_signals(data):
    
    timestamp = data['Timestamp']
    partition = data['Partition']
    off_set = data['Offset']
    Message = data['Message']
    moment = Message['moment']
    data = [timestamp, partition, off_set, moment] + list(Message.values())[1:]
    cursor.execute(f"INSERT INTO blm.moment_signals VALUES ({str(data)[1:-1]});")
    connect.commit()

In [202]:
#len(example_kafka)

In [203]:
# запись данных в таблицу 'moment_signals'
#%%time
#for example in example_kafka[1:]:
#    insert_moment_signals(example)

In [204]:
# проверка записи данных в таблицу 'moment_signals'
#cursor.execute("SELECT COUNT(*) FROM blm.moment_signals;")
#cursor.fetchone()

In [205]:
# вывод наименований таблиц схемы 'Bl model' - Storage Layer
cursor.execute("SELECT table_name FROM information_schema.tables \
                WHERE table_schema = 'blm';")
cursor.fetchall()

[('measurings',),
 ('mapping_signals',),
 ('calendar',),
 ('moment_signals',),
 ('signal_settings',),
 ('exhausters',),
 ('mechanisms',),
 ('signals',),
 ('marks',)]

# 5. ETL 'Stagen' & 'Bl model' - Storage Layer

In [ ]:
#connect, cursor = connect_db(DATABASE_URL)

In [345]:
# создание схемы 'Bl model' - Storage Layer
#cursor3.execute("CREATE SCHEMA blm;")
#connect3.commit()  

In [26]:
# удаление таблицы 'mechanisms' если она существует 
#cursor3.execute("DROP TABLE IF EXISTS blm.moment_signals;")
#connect3.commit() 

In [32]:
# запрос на  создание таблицы - 'moment_signals'
#query = f"""CREATE TABLE blm.moment_signals( \
#                 timestamp TEXT PRIMARY KEY, \
#                 partition TEXT NOT NULL, \
#                 off_set TEXT NOT NULL, \
#                 moment DATE NOT NULL, {", ".join(map(str, SM_Exgauster))});"""

In [33]:
# создание таблицы - 'moment_signals'
#cursor3.execute(query)
#connect3.commit()

In [206]:
# значение атрибутов в таблице 'moment_signals'
#cursor3.execute("SELECT column_name \
#             FROM information_schema.columns \
#             WHERE table_name = 'moment_signals' AND table_schema = 'blm';")
#columnsname = [_[0] for _ in cursor3.fetchall()]
#print(columnsname)

In [207]:
# данные для записи в таблицу 'measurings'
#cursor2.execute("SELECT * FROM stg.signal_messages;")
#rows = cursor2.fetchall()

In [23]:
# функция записи данных в таблицу 'measurings'
def insert_moment_signals(data):
    
    timestamp = data[0][3]
    partition = data[0][1]
    off_set = data[0][-2]
    Message = data[0][6]
    moment = Message['moment']
    data = [timestamp, partition, off_set, moment] + list(Message.values())[1:]
    cursor3.execute(f"INSERT INTO blm.moment_signals VALUES ({str(data)[1:-1]});")
    connect3.commit()

In [208]:
# запись данных в таблицу 'measurings'
#%%time
#for row in rows:
#    try:
#        insert_moment_signals(row)
#    except:
#        print('Закончилось место в облаке')
#        break

In [209]:
# проверка записи данных в таблицу 'measurings'
#cursor3.execute("SELECT COUNT(*) FROM blm.moment_signals;")
#cursor3.fetchone()

In [234]:
# закрытие подключения к базе данных 'postgres'
connect.close()
#connect2.close()
#connect3.close()